**Connect google drive**

In [17]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Jan  3 12:55:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
!pip install numpy # type: ignore

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 8498FEC4-2262-42A9-A0DA-8C599C76A6E5.png
'8498FEC4-2262-42A9-A0DA-8C599C76A6E5 - Seif El Din Seif El Din Ahmed Mohamed Fouad Zaki.png'
'Colab Notebooks'
'Getting started.pdf'
'heuristic testing.gdoc'
'My Drive'
'On-campus robot.gslides'
 Packages.gform
'Pick & Go.zip'
'Restaurants .gform'
'Sharm el sheikh college.gform'
 Smoking
'source code'
 Untitled0.ipynb
'Untitled form (1).gform'
'Untitled form.gform'
 yolov3
 yolov3_training_last.weights


**1) Clone the Darknet**



In [20]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [21]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
     

**3) Configure Darknet network for training YOLO V3**

In [22]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [23]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [24]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [25]:
!echo "smoking" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [26]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-01-03 12:56:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  56.1MB/s    in 2.8s    

2022-01-03 12:56:53 (56.1 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [27]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/smoke0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/smoke0.jpg     
  inflating: data/obj/smoke0.txt     
  inflating: data/obj/smoke1.jpg     
  inflating: data/obj/smoke1.txt     
  inflating: data/obj/smoke2.jpg     
  inflating: data/obj/smoke2.txt     
  inflating: data/obj/smoke3.jpg     
  inflating: data/obj/smoke3.txt     
  inflating: data/obj/smoke4.jpg     
  inflating: data/obj/smoke4.txt     
  inflating: data/obj/smoke5.jpg     
  inflating: data/obj/smoke5.txt     
  inflating: data/obj/smoke6.jpg     
  inflating: data/obj/smoke6.txt     
  inflating: data/obj/smoke7.jpg     
  inflating: data/obj/smoke7.txt     
  inflating: data/obj/smoke8.jpg     
  inflating: data/obj/smoke8.txt     
  inflating: data/obj/smoke9.jpg     
  inflating: data/obj/smoke9.txt     
  inflating: data/obj/smoke10.jpg    
  inflating: data/obj/smoke10.txt    
  inflating: data/obj/smoke11.jpg    
  inflating: data/ob

In [28]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

dog

[]
person

[]
cat

[]
tv

[]
car

[]
meatballs

[]
marinara sauce

[]
tomato soup

[]
chicken noodle soup

[]
french onion soup

[]
chicken breast

[]
ribs

[]
pulled pork

[]
hamburger

[]
cavity

[]
smoking

[]
15 0.701309 0.541295 0.191489 0.328125

['15', '0.701309', '0.541295', '0.191489', '0.328125']
1 data/obj/smoke82.txt
0 0.701309 0.541295 0.191489 0.328125
15 0.378000 0.834835 0.492000 0.306306

['15', '0.378000', '0.834835', '0.492000', '0.306306']
2 data/obj/smoke0.txt
0 0.378000 0.834835 0.492000 0.306306
15 0.588379 0.532258 0.743164 0.803519

['15', '0.588379', '0.532258', '0.743164', '0.803519']
4 data/obj/smoke34.txt
0 0.588379 0.532258 0.743164 0.803519
15 0.522000 0.728333 0.400000 0.376667

['15', '0.522000', '0.728333', '0.400000', '0.376667']
5 data/obj/smoke36.txt
0 0.522000 0.728333 0.400000 0.376667
15 0.569052 0.753687 0.409318 0.297935

['15', '0.569052', '0.753687', '0.409318', '0.297935']
6 data/obj/smoke11.txt
0 0.569052 0.753687 0.409318 0.297935
15 

In [29]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/smoke22.jpg', 'data/obj/smoke59.jpg', 'data/obj/smoke13.jpg', 'data/obj/smoke85.jpg', 'data/obj/smoke69.jpg', 'data/obj/smoke68.jpg', 'data/obj/smoke46.jpg', 'data/obj/smoke4.jpg', 'data/obj/smoke63.jpg', 'data/obj/smoke70.jpg', 'data/obj/smoke64.jpg', 'data/obj/smoke8.jpg', 'data/obj/smoke56.jpg', 'data/obj/smoke3.jpg', 'data/obj/smoke35.jpg', 'data/obj/smoke71.jpg', 'data/obj/smoke45.jpg', 'data/obj/smoke19.jpg', 'data/obj/smoke65.jpg', 'data/obj/smoke44.jpg', 'data/obj/smoke49.jpg', 'data/obj/smoke23.jpg', 'data/obj/smoke42.jpg', 'data/obj/smoke83.jpg', 'data/obj/smoke75.jpg', 'data/obj/smoke41.jpg', 'data/obj/smoke58.jpg', 'data/obj/smoke28.jpg', 'data/obj/smoke39.jpg', 'data/obj/smoke31.jpg', 'data/obj/smoke30.jpg', 'data/obj/smoke32.jpg', 'data/obj/smoke10.jpg', 'data/obj/smoke55.jpg', 'data/obj/smoke36.jpg', 'data/obj/smoke51.jpg', 'data/obj/smoke17.jpg', 'data/obj/smoke67.jpg', 'data/obj/smoke86.jpg', 'data/obj/smoke81.jpg', 'data/obj/smoke73.jpg', 'data/obj/smoke26.

In [30]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
print("gmed")
file.close() 


gmed


**6) Start the training**

In [32]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
#!./darknet detector test cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show -ext_output < data/train.txt > result.txt

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
^C
